In [ ]:
# Name - Md shahriar rashid rahi
# Id : 221-35-1027
# Sec: E(37)

In [ ]:
# Loading the required packages and libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 1. Loading data
import pandas as pd
df = pd.read_csv('/content/train ai.csv')
df_test = pd.read_csv('/content/test Ai.csv')

# Replacing with mean for LotFrontage
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].mean())
df_test['LotFrontage'] = df_test['LotFrontage'].fillna(df_test['LotFrontage'].mean())

# Dropping columns with a lot of missing values
columns_to_drop = ['PoolQC', 'Fence', 'MiscFeature', 'FireplaceQu']
df.drop(columns=columns_to_drop, axis=1, inplace=True)
df_test.drop(columns=columns_to_drop, axis=1, inplace=True)

In [ ]:
# 2. Data Exploration
df.head()
df_test.shape
df.shape
# Checking the null values
df.isnull().sum()
df_test.isnull().sum()

,0
Id,0
MSSubClass,0
MSZoning,4
LotFrontage,0
LotArea,0
...,...
MiscVal,0
MoSold,0
YrSold,0
SaleType,1


In [ ]:
# 3. Data Cleaning (Handling missing values)
# Handling missing data for MSZoning
df_test['MSZoning'] = df_test['MSZoning'].fillna(df_test['MSZoning'].mode()[0])

In [ ]:
# 4. Data Transformation (Converting categorical variables to one-hot encoding)
def category_onehot_multcols(multcolumns):
    df_final = final_df
    i = 0
    for fields in multcolumns:
        df1 = pd.get_dummies(final_df[fields], drop_first=True)
        final_df.drop([fields], axis=1, inplace=True)
        if i == 0:
            df_final = df1.copy()
        else:
            df_final = pd.concat([df_final, df1], axis=1)
        i += 1
    df_final = pd.concat([final_df, df_final], axis=1)
    return df_final

# Concatenating test and train datasets
main_df = df.copy()
final_df = pd.concat([df, df_test], axis=0)

# Transform categorical variables
columns = final_df.select_dtypes(include=['object']).columns
final_df = category_onehot_multcols(columns)

# Removing duplicated columns
final_df = final_df.loc[:, ~final_df.columns.duplicated()]

# Separating the test and training data
df_Train = final_df.iloc[:1459, :]
df_Test = final_df.iloc[1459:, :]

# Dropping "SalePrice" from test data
df_Test.drop(['SalePrice'], axis=1, inplace=True)

# Preparing data for the model
X_train = df_Train.drop(['SalePrice'], axis=1)
y_train = df_Train['SalePrice']

<ipython-input-13-502199ab08cb>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Test.drop(['SalePrice'], axis=1, inplace=True)


In [ ]:
# 5. Base Model Performance (XGBoost)

import xgboost as xgb
classifier = xgb.XGBRegressor()
classifier.fit(X_train, y_train)

# Predicting the house prices
y_predict = classifier.predict(df_Test)

# Creating a submission file
pred = pd.DataFrame(y_predict)
sub_df = pd.read_csv('sample_submission.csv')
datasets = pd.concat([sub_df['Id'], pred], axis=1)
datasets.columns = ['Id', 'SalePrice']
datasets.to_csv('sub1.csv', index=False)

In [ ]:
# 6. Hyperparameter Tuning
from sklearn.model_selection import RandomizedSearchCV

regressor = xgb.XGBRegressor()

# Defining the grid of hyperparameters
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
learning_rate = [0.05, 0.1, 0.15, 0.20]
min_child_weight = [1, 2, 3, 4]
booster = ['gbtree', 'gblinear']
base_score = [0.25, 0.5, 0.75, 1]

hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'learning_rate': learning_rate,
    'min_child_weight': min_child_weight,
    'booster': booster,
    'base_score': base_score
}

# Random search with cross-validation
random_cv = RandomizedSearchCV(
    estimator=regressor,
    param_distributions=hyperparameter_grid,
    cv=5, n_iter=50,
    scoring='neg_mean_absolute_error', n_jobs=4,
    verbose=5, return_train_score=True,
    random_state=42
)

random_cv.fit(X_train, y_train)
random_cv.best_estimator_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


XGBRegressor(base_score=0.25, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.15, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [16]:
# 7. Final Model
best_params = random_cv.best_params_
regressor = xgb.XGBRegressor(**best_params)
regressor.fit(X_train, y_train)

# Predicting with the tuned model
y_predict1 = regressor.predict(df_Test)

# Creating the final submission file
pred = pd.DataFrame(y_predict1)
datasets = pd.concat([sub_df['Id'], pred], axis=1)
datasets.columns = ['Id', 'SalePrice']
datasets.to_csv('housepricetuned.csv', index=False)